In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score

In [2]:
df = pd.read_csv('data.csv')

In [3]:
df = df.drop(['Unnamed: 0'], axis=1)

In [4]:
df["Date"] = pd.to_datetime(df["Date"]).dt.strftime("%Y%m%d")

In [10]:
df['y/m'] = pd.to_datetime(df['y/m']).dt.strftime("%Y%m%d")

In [12]:
X = df.drop(['3_sigma_event'], axis=1).values
Y = df['3_sigma_event'].values

print(X.shape, Y.shape)

(7345, 49) (7345,)


In [13]:
nrows=7345
n_train = int(nrows * 0.9)
X_train = X[:n_train]
Y_train = Y[:n_train]
X_test = X[n_train:]
Y_test = Y[n_train:]

In [14]:
import tensorflow as tf
from tensorflow.contrib.tensor_forest.python import tensor_forest
from tensorflow.python.ops import resources

C:\Users\Thier\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [15]:
n_iter = 50
n_classes = 2
n_features = int(X_train.shape[1])
n_trees = 10
max_nodes = 30000

In [16]:
x = tf.placeholder(tf.float32, shape=[None, n_features])
y = tf.placeholder(tf.int64, shape=[None])

In [17]:
hparams = tensor_forest.ForestHParams(num_classes=n_classes, num_features=n_features, num_trees=n_trees,
                                      max_nodes=max_nodes, split_after_samples=50).fill()

In [18]:
forest_graph = tensor_forest.RandomForestGraphs(hparams)

INFO:tensorflow:Constructing forest with params = 
INFO:tensorflow:{'num_trees': 10, 'max_nodes': 30000, 'bagging_fraction': 1.0, 'feature_bagging_fraction': 1.0, 'num_splits_to_consider': 10, 'max_fertile_nodes': 0, 'split_after_samples': 50, 'valid_leaf_threshold': 1, 'dominate_method': 'bootstrap', 'dominate_fraction': 0.99, 'model_name': 'all_dense', 'split_finish_name': 'basic', 'split_pruning_name': 'none', 'collate_examples': False, 'checkpoint_stats': False, 'use_running_stats_method': False, 'initialize_average_splits': False, 'inference_tree_paths': False, 'param_file': None, 'split_name': 'less_or_equal', 'early_finish_check_every_samples': 0, 'prune_every_samples': 0, 'num_classes': 2, 'num_features': 49, 'bagged_num_features': 49, 'bagged_features': None, 'regression': False, 'num_outputs': 1, 'num_output_columns': 3, 'base_random_seed': 0, 'leaf_model_type': 0, 'stats_model_type': 0, 'finish_type': 0, 'pruning_type': 0, 'split_type': 0}


In [19]:
train_op = forest_graph.training_graph(x, y)
loss_op = forest_graph.training_loss(x, y)

In [20]:
infer_op, _, _ = forest_graph.inference_graph(x)

In [21]:
auc = tf.metrics.auc(tf.cast(y, tf.int64), infer_op[:, 1])[1]

In [22]:
init_vars = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer(), resources.initialize_resources(resources.shared_resources()))

sess = tf.Session()

sess.run(init_vars)

In [23]:
batch_size = 1000

In [24]:
indices = list(range(n_train))

In [25]:
def gen_batch(indices):
    np.random.shuffle(indices)
    for batch_i in range(int(n_train / batch_size)):
        batch_index = indices[batch_i*batch_size: (batch_i+1)*batch_size]
        yield X_train[batch_index], Y_train[batch_index]

In [26]:
for i in range(1, n_iter + 1):
    for X_batch, Y_batch in gen_batch(indices):
        _, l = sess.run([train_op, loss_op], feed_dict={x: X_batch, y: Y_batch})
    acc_train = sess.run(auc, feed_dict={x: X_train, y: Y_train})
    print('Iteration %i, AUC of ROC on training set: %f' % (i, acc_train))
    acc_test = sess.run(auc, feed_dict={x: X_test, y: Y_test})
    print("AUC of ROC on testing set:", acc_test)

Iteration 1, AUC of ROC on training set: 0.968983
AUC of ROC on testing set: 0.9608021
Iteration 2, AUC of ROC on training set: 0.976716
AUC of ROC on testing set: 0.9731716
Iteration 3, AUC of ROC on training set: 0.980440
AUC of ROC on testing set: 0.95857066
Iteration 4, AUC of ROC on training set: 0.966436
AUC of ROC on testing set: 0.94384336
Iteration 5, AUC of ROC on training set: 0.950830
AUC of ROC on testing set: 0.9397519
Iteration 6, AUC of ROC on training set: 0.946034
AUC of ROC on testing set: 0.9370217
Iteration 7, AUC of ROC on training set: 0.943244
AUC of ROC on testing set: 0.9356367
Iteration 8, AUC of ROC on training set: 0.941025
AUC of ROC on testing set: 0.9309618
Iteration 9, AUC of ROC on training set: 0.936091
AUC of ROC on testing set: 0.93084586
Iteration 10, AUC of ROC on training set: 0.935603
AUC of ROC on testing set: 0.93090254
Iteration 11, AUC of ROC on training set: 0.935306
AUC of ROC on testing set: 0.9310774
Iteration 12, AUC of ROC on training 

In [25]:
#With no feature engineering, we achieved a ROCAUC of almost 80